In [10]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

data1 = load_dataset('Howard881010/gas-3week')
# data2 = load_dataset('howard881010/climate-3day-mixed')
# data3 = load_dataset('howard881010/climate-2day')
# data4 = load_dataset('howard881010/climate-2day-mixed')
# data5 = load_dataset('howard881010/climate-1day')
# data6 = load_dataset('howard881010/climate-1day-mixed')

# data_all = {}

# for split in ['train', 'test', 'valid']:
#     data_all[split] = concatenate_datasets([data1[split], data2[split], data3[split], data4[split], data5[split], data6[split]])

# data_dict = DatasetDict({
#     'train': data_all['train'],
#     'test': data_all['test'],
#     'valid': data_all['valid']
# })

# data_dict.push_to_hub('Howard881010/climate')
# print(data1['test']['instruction'][0])

Generating train split:   0%|          | 0/1293 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/162 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/162 [00:00<?, ? examples/s]

In [11]:
print(data1['test']['instruction'][0])
print(data1['test']['input'][0])

Given the gas related facts of the first 3 week, predict the gas related facts of the next 3 week.
Output the result in the following format:
```
week_4_date:
week_4_gas_news:
week_5_date:
week_5_gas_news:
week_6_date:
week_6_gas_news:
```
```
week_1_date: 2020-01-27
week_1_gas_news: The air quality in New York City reached unhealthy levels due to incoming smoke from wildfires and agricultural burning in Canada, and the city's most polluted areas are Northern Manhattan and the South Bronx.
week_2_date: 2020-02-03
week_2_gas_news: As of February 2020, air quality monitoring stations are available in Public School 19, New York, USA, measuring PM2.5 and PM10 in real-time.
week_3_date: 2020-02-10
week_3_gas_news: New York City's neighborhoods have varying air quality, with some areas experiencing high levels of air pollution.
```
